# Preprocessing #7.1

##### Data version: Drop all OHE
##### Model version: regularized Decision Tree

In [191]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [192]:
# import data

df = pd.read_csv('../data/train.csv')

In [193]:
# check data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Data Cleaning---

In [194]:
# Converting integers to string for the OrdinalEncoder

def convert_int_to_string(data, columns):
  for column in columns:
    data[column] = data[column].astype(str)
  return data

In [195]:
# Define features to convert and convert

columns_to_convert = ['OverallQual', 'OverallCond']

df = convert_int_to_string(df, columns_to_convert)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   object 
 18  OverallC

In [196]:
# OutlierRemover is a custom transformer imported from src/helpers.py
# It can remove outliers from multiple columns by specifying the iqr_multipler

outlier_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea',
                'OpenPorchSF']

outlier_remover = OutlierRemover(columns=outlier_cols,iqr_multiplier=5)

outlier_remover.fit(df)
df_proc = outlier_remover.transform(df)

df_proc.count()

Id               1428
MSSubClass       1428
MSZoning         1428
LotFrontage      1178
LotArea          1428
                 ... 
MoSold           1428
YrSold           1428
SaleType         1428
SaleCondition    1428
SalePrice        1428
Length: 81, dtype: int64

# ---Pipeline---

In [197]:
# defining all OHE columns first, to add them to the columns to remove filter, to see how model performs
# with all the OHE features remove, as an attempt to reduce noise and decrease overfitting

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

In [198]:
# Defining the list of columns that will go into the pipeline
# Remove columns that should not be included in the pipeline

all_columns = df.columns.tolist()
columns_remove_from_filter = ['SalePrice','Id','Street','Utilities','LowQualFinSF','GarageYrBlt',
                              '3SsnPorch']

# Add the OHE columns to the remove filter
columns_remove_from_filter.extend(ohe_cols)

columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [199]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

In [200]:
# Preprocessor for imputing all null variables

# separating the OHE and OE columns to remove OHE features from the preprocessor
cols_impute_na_ohe = ['Alley', 'GarageType', 'MiscFeature']
cols_impute_na_oe = ['Fence','BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2', 'FireplaceQu',
                     'GarageFinish', 'GarageQual','GarageCond','PoolQC']

cols_impute_na = cols_impute_na_oe
cols_impute_none = ['MasVnrType']
cols_impute_most_freq = ['Electrical']
cols_impute_zero = ['LotFrontage', 'MasVnrArea']

preprocessor_imputer = ColumnTransformer([
  ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),

  # disabling features that are OHE
  # ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),

  ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),

  # disabling features that are OHE
  # ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)

], remainder='passthrough', verbose_feature_names_out=False)

In [201]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_1_10 = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'OverallQual', 'mapping':dict_1_10},
  {'col': 'OverallCond', 'mapping':dict_1_10},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

In [202]:
# Preprocessor for encoding and scaling

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']

robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
               'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal']

preprocessor_encode_scale = ColumnTransformer([
  #disabling the OHE to drop all features using OHE
  # ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),
  ('oe', OrdinalEncoder(), oe_cols),
  ('standard_scaler', StandardScaler(), ['LotFrontage','GarageArea']),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), ['YearBuilt','YearRemodAdd'])
], remainder='passthrough', verbose_feature_names_out=False)

In [203]:
# split train and val

X = df_proc.drop(columns='SalePrice')
y = df_proc['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [204]:
# run the main pipeline with all preprocessors

main_pipe = Pipeline([
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [205]:
# fit and transform X_train and X_val

main_pipe.fit(X_train)

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

In [206]:
# checking the number of columns in X_train after processing

len(X_train_proc.columns)

45

### Decision Tree Regressor

In [207]:
# Import, fit on model, then save results

from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train_proc, y_train)

linear_results_train = evaluate_regression(dt, X_train_proc, y_train, 'dt depth=23')
linear_results_val = evaluate_regression(dt, X_val_proc, y_val, 'dt depth=23')

In [208]:
# check the depth

dt.get_depth()

23

In [209]:
dt = DecisionTreeRegressor(max_depth=21)
dt.fit(X_train_proc, y_train)


linear_results_train = pd.concat([linear_results_train, evaluate_regression(dt, X_train_proc, y_train, 'dt depth=21')])
linear_results_val = pd.concat([linear_results_val, evaluate_regression(dt, X_val_proc, y_val, 'dt depth=21')])

display(linear_results_train)
display(linear_results_val)

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
dt depth=21,5.896089,5872.737887,76.633791,0.000031,0.999999,0.999999


,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,28272.814685,1.861229e+09,43141.967761,0.173534,0.699796,0.643508
dt depth=21,27418.920746,1.811167e+09,42557.810583,0.169233,0.707871,0.653097


In [210]:
dt = DecisionTreeRegressor(max_depth=20)
dt.fit(X_train_proc, y_train)


linear_results_train = pd.concat([linear_results_train, evaluate_regression(dt, X_train_proc, y_train, 'dt depth=20')])
linear_results_val = pd.concat([linear_results_val, evaluate_regression(dt, X_val_proc, y_val, 'dt depth=20')])

display(linear_results_train)
display(linear_results_val)

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
dt depth=21,5.896089,5872.737887,76.633791,0.000031,0.999999,0.999999
dt depth=20,17.483446,39053.455675,197.619472,0.000092,0.999993,0.999993


,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,28272.814685,1.861229e+09,43141.967761,0.173534,0.699796,0.643508
dt depth=21,27418.920746,1.811167e+09,42557.810583,0.169233,0.707871,0.653097
dt depth=20,28068.487179,1.795272e+09,42370.645535,0.174376,0.710435,0.656141


In [211]:
dt = DecisionTreeRegressor(max_depth=15)
dt.fit(X_train_proc, y_train)


linear_results_train = pd.concat([linear_results_train, evaluate_regression(dt, X_train_proc, y_train, 'dt depth=15')])
linear_results_val = pd.concat([linear_results_val, evaluate_regression(dt, X_val_proc, y_val, 'dt depth=15')])

display(linear_results_train)
display(linear_results_val)

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,0.000000,0.000000e+00,0.000000,0.000000,1.000000,1.000000
dt depth=21,5.896089,5.872738e+03,76.633791,0.000031,0.999999,0.999999
dt depth=20,17.483446,3.905346e+04,197.619472,0.000092,0.999993,0.999993
dt depth=15,564.067064,4.821485e+06,2195.787946,0.003580,0.999148,0.999113


,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,28272.814685,1.861229e+09,43141.967761,0.173534,0.699796,0.643508
dt depth=21,27418.920746,1.811167e+09,42557.810583,0.169233,0.707871,0.653097
dt depth=20,28068.487179,1.795272e+09,42370.645535,0.174376,0.710435,0.656141
dt depth=15,30000.317607,1.967469e+09,44356.156853,0.188689,0.682660,0.623159


In [212]:
dt = DecisionTreeRegressor(max_depth=10)
dt.fit(X_train_proc, y_train)


linear_results_train = pd.concat([linear_results_train, evaluate_regression(dt, X_train_proc, y_train, 'dt depth=10')])
linear_results_val = pd.concat([linear_results_val, evaluate_regression(dt, X_val_proc, y_val, 'dt depth=10')])

display(linear_results_train)
display(linear_results_val)

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,0.000000,0.000000e+00,0.000000,0.000000,1.000000,1.000000
dt depth=21,5.896089,5.872738e+03,76.633791,0.000031,0.999999,0.999999
dt depth=20,17.483446,3.905346e+04,197.619472,0.000092,0.999993,0.999993
dt depth=15,564.067064,4.821485e+06,2195.787946,0.003580,0.999148,0.999113
dt depth=10,5881.067172,9.046205e+07,9511.154160,0.035770,0.984015,0.983359


,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,28272.814685,1.861229e+09,43141.967761,0.173534,0.699796,0.643508
dt depth=21,27418.920746,1.811167e+09,42557.810583,0.169233,0.707871,0.653097
dt depth=20,28068.487179,1.795272e+09,42370.645535,0.174376,0.710435,0.656141
dt depth=15,30000.317607,1.967469e+09,44356.156853,0.188689,0.682660,0.623159
dt depth=10,27065.471927,1.948788e+09,44145.078954,0.172612,0.685673,0.626737


In [213]:
dt = DecisionTreeRegressor(max_depth=8)
dt.fit(X_train_proc, y_train)


linear_results_train = pd.concat([linear_results_train, evaluate_regression(dt, X_train_proc, y_train, 'dt depth=8')])
linear_results_val = pd.concat([linear_results_val, evaluate_regression(dt, X_val_proc, y_val, 'dt depth=8')])

display(linear_results_train)
display(linear_results_val)

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,0.000000,0.000000e+00,0.000000,0.000000,1.000000,1.000000
dt depth=21,5.896089,5.872738e+03,76.633791,0.000031,0.999999,0.999999
dt depth=20,17.483446,3.905346e+04,197.619472,0.000092,0.999993,0.999993
dt depth=15,564.067064,4.821485e+06,2195.787946,0.003580,0.999148,0.999113
dt depth=10,5881.067172,9.046205e+07,9511.154160,0.035770,0.984015,0.983359
dt depth=8,11098.667350,2.395217e+08,15476.488419,0.066821,0.957677,0.955939


,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt depth=23,28272.814685,1.861229e+09,43141.967761,0.173534,0.699796,0.643508
dt depth=21,27418.920746,1.811167e+09,42557.810583,0.169233,0.707871,0.653097
dt depth=20,28068.487179,1.795272e+09,42370.645535,0.174376,0.710435,0.656141
dt depth=15,30000.317607,1.967469e+09,44356.156853,0.188689,0.682660,0.623159
dt depth=10,27065.471927,1.948788e+09,44145.078954,0.172612,0.685673,0.626737
dt depth=8,26759.003468,1.843899e+09,42940.641757,0.166812,0.702591,0.646827


In [215]:
# calculating the log RMSE
# optimal depth = 20

dt = DecisionTreeRegressor(max_depth=20)
dt.fit(X_train_proc, y_train)

val_preds = dt.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.23490091525770174